In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("/home/florian/bachelorarbeit/code/Cross-Silo-FL/datasets/horizontal/covid/owid-covid-data.csv")

In [48]:
data.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-02-24,5.0,5.0,NaN,NaN,NaN,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-02-25,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-02-26,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-02-27,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-02-28,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN


In [8]:
import pickle
from pathlib import Path

#loading covid data
from sktime.forecasting.model_selection import SlidingWindowSplitter
from sklearn.preprocessing import StandardScaler


def _get_samples_from_covid_data(n: int, attributes: list[str], num_of_samples: int, serialize: bool):
    """
    Generates samples from the covid dataset.
    Args:
        n (int): Numbers of records per sample.
        attributes (list[str]): List of attributes that will be used from the dataset. The fist element is the endogene variable.
        num_of_samples (int): Number of returned samples.
    """
    
    #load data
    data = pd.read_csv("../datasets/horizontal/covid/owid-covid-data.csv")

    #load data if already serialized
    path = Path(f'../datasets/samples/covid_{n}_{"_".join(attributes)}.pkl')
    if path.exists():
        pkl_file = open(path, 'rb')
        x_data, y_data = pickle.load(pkl_file)
        pkl_file.close()
        
        if len(data.index) > num_of_samples:
            return x_data[:num_of_samples], y_data[:num_of_samples]

        else:
            return x_data[:len(data.index) - 1], y_data[:len(data.index) - 1]

    
    #scale the data
    record_info = data[["iso_code", "continent", "location", "date", "tests_units"]]
    data = data.drop(["iso_code", "continent", "location", "date", "tests_units"], axis=1)
    data_columns = data.columns
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(data)
    data = pd.DataFrame(scaled_data, columns=data_columns)
    data = pd.concat([data, record_info], axis=1)

    
    x_data = []
    y_data = []

    #split the data
    countries = data.iso_code.drop_duplicates(keep="first")
    countries = countries[countries != "ESH"] #drop ESH because it only has one entry

    for country in countries:
        country_data = data[data.iso_code == country]

        splitter = SlidingWindowSplitter(fh=1, window_length=n)
        samples = splitter.split_series(country_data[attributes].to_numpy())

        for sample in samples:
            x_sample = sample[0].flatten()
            y_sample = sample[1].flatten()[attributes.index("new_cases")]

            if not np.isnan(np.sum(x_sample)) and not np.isnan(y_sample): #check for nans
                    x_data.append(x_sample)
                    y_data.append(y_sample)
   
    #save data
    if serialize:
            output = open(path, "wb")
            pickle.dump((x_data, y_data), output)
            output.close()

    #enusre that sample number is not out of range
    if len(data.index) > num_of_samples:
        return x_data[:num_of_samples], y_data[:num_of_samples]
    else:
        return x_data[:len(data.index) - 1], y_data[:len(data.index) - 1]


In [21]:
test_samples = _get_samples_from_covid_data(10, ["new_cases"], 100, True)

In [22]:
test_samples[0][:3]

[array([-0.1447642 , -0.14482342, -0.14482342, -0.14482342, -0.14482342,
        -0.14482342, -0.14482342, -0.14482342, -0.14482342, -0.14482342]),
 array([-0.14482342, -0.14482342, -0.14482342, -0.14482342, -0.14482342,
        -0.14482342, -0.14482342, -0.14482342, -0.14482342, -0.14482342]),
 array([-0.14482342, -0.14482342, -0.14482342, -0.14482342, -0.14482342,
        -0.14482342, -0.14482342, -0.14482342, -0.14482342, -0.14482342])]

In [23]:
test_samples[1][:3]

[-0.14482341997863235, -0.14482341997863235, -0.14478789054959926]

_Weather data starts here_

In [2]:
data = pd.read_csv("/home/florian/bachelorarbeit/code/Cross-Silo-FL/datasets/vertical/weather/berlin_alexanderplatz.csv", names=["time", "temp", "dwpt", "rhum", "prcp", "snow", "wdir", "wspd", "wpgt", "pres", "tsun", "coco"])
data.describe()

,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco
count,16515.000000,16513.000000,16514.000000,11987.000000,4117.000000,10598.000000,11034.000000,8395.000000,11936.000000,31.000000,0.0
mean,10.608653,7.365076,14.120358,1.579361,5.176099,208.351670,28.778204,53.423204,1010.997009,15.483871,NaN
std,7.861329,6.997318,8.975399,3.642694,23.663484,93.444179,15.034219,21.768710,9.549732,60.651942,NaN
min,-17.600000,-18.700000,-14.900000,0.000000,0.000000,0.000000,0.000000,0.000000,961.300000,0.000000,NaN
25%,4.600000,2.200000,7.100000,0.000000,0.000000,131.000000,16.600000,36.000000,1005.200000,0.000000,NaN
50%,10.600000,7.400000,14.100000,0.000000,0.000000,235.000000,26.600000,51.500000,1011.300000,0.000000,NaN
75%,16.800000,13.000000,21.100000,1.500000,0.000000,280.000000,38.900000,67.700000,1017.200000,0.000000,NaN
max,31.100000,24.700000,38.000000,60.400000,310.000000,360.000000,115.200000,154.800000,1044.800000,276.000000,NaN


In [3]:

import sys
sys.path.insert(1, '../scripts')
import helper

2023-01-26 15:40:54.771133: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-26 15:40:56.375555: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-26 15:40:56.377150: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-01-26 15:40:56.377170: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [9]:
from sktime.forecasting.model_selection import SlidingWindowSplitter


def _get_samples_from_weather_data(n: int, attributes: list[str], station: str, num_of_samples: int):
    """
    Generates samples from the covid dataset.
    Args:
        n (int): Numbers of records per sample.
        attributes (list[str]): List of attributes that will be used from the dataset. The fist element is the endogene variable.
        station (str): Name of the station.
        num_of_samples (int): Number of returned samples.
    """



    #load data
    path = f"/home/florian/bachelorarbeit/code/Cross-Silo-FL/datasets/vertical/weather/{station}.csv"
    data = pd.read_csv(path, names=["time", "temp", "dwpt", "rhum", "prcp", "snow", "wdir", "wspd", "wpgt", "pres", "tsun", "coco"])
    
    #scale the data
    data = data.drop("time", axis=1)
    data_columns = data.columns
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(data)
    data = pd.DataFrame(scaled_data, columns=data_columns)

    #split the data
    splitter = SlidingWindowSplitter(fh=1, window_length=n)
    samples = splitter.split_series(data[attributes].to_numpy())

    x_data = []
    y_data = []

    for sample in samples:
        x_sample = sample[0].flatten()
        y_sample = sample[1].flatten()[0] #the endogene temperature variable

        if not np.isnan(np.sum(x_sample)) and not np.isnan(y_sample): #check for nans
            x_data.append(x_sample)
            y_data.append(y_sample)

    return x_data[:num_of_samples], y_data[:num_of_samples]



In [10]:
x_train, y_train = _get_samples_from_weather_data(10, ["temp"], "berlin_alexanderplatz", 10000)

/home/florian/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:1047: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/home/florian/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:1052: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/home/florian/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:1072: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


In [6]:
pd_x = pd.DataFrame(x_train)
pd_x.describe()
x_train[0]

NameError: name 'x_train' is not defined

In [70]:
pd_y = pd.Series(y_train)
pd_y.describe()
y_train[0]

-1.285909696369293

In [71]:
old_x_train[0]

array([-1.3,  1.5, -0.5, -3.2, -2.1, -2.4, -3.2, -3.8, -3.4,  0.4])

In [2]:
import tensorflow as tf

2023-01-03 18:59:26.252891: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-03 18:59:26.695633: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-03 18:59:26.695667: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-03 18:59:27.812762: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [3]:
path = "/home/florian/bachelorarbeit/code/Cross-Silo-FL/datasets/vertical/weather/muenchen.csv"

In [4]:
data = pd.read_csv(path, names=["time", "temp", "dwpt", "rhum", "prcp", "snow", "wdir", "wspd", "wpgt", "pres", "tsun", "coco"])
data.head()

,time,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco
0,1879-01-01,NaN,4.6,9.8,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1879-01-02,NaN,0.7,8.5,2.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1879-01-03,NaN,-0.9,8.6,0.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1879-01-04,NaN,0.4,8.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1879-01-05,NaN,-4.7,0.0,5.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
exog_columns = ["temp", "pres", "tsun"]
edog_column = "temp"

In [9]:
tf_dataset = tf.data.Dataset.from_tensor_slices(dict(data))

In [55]:
exog = None
edog = None
x_train = []
y_train = []
for window in tf_dataset.batch(11, drop_remainder=True).take(10000):
    new_df = pd.DataFrame(window)
    exog = new_df[exog_columns].iloc[:10]
    edog = new_df["time"].iloc[-1]

    if not exog.isnull().values.any() or not edog: #check for NaNs
        exog = exog.to_numpy().flatten()


        x_train.append(exog)
        y_train.append(edog)
    #     exog = None
    #     edog = None

print(y_train[0:2])

[b'1985-01-15', b'1985-01-26']


## LSTM model

In [17]:
samples = helper.get_samples("weather", 10, ["temp"], "potsdam", True, 10000)

In [16]:
import tensorflow as tf

In [38]:
model = tf.keras.Sequential()
input_shape = np.array(samples[0]).shape[1]

#add layers
model.add(tf.keras.layers.Input(shape=(input_shape,)))
model.add(tf.keras.layers.Conv1D(32, kernel_size=(10,)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64))
#output layer
model.add(tf.keras.layers.Dense(1))

model.compile(optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.MeanAbsoluteError(),
        metrics=["mean_absolute_error"])

model.summary()

ValueError: Input 0 of layer "conv1d_8" is incompatible with the layer: expected min_ndim=3, found ndim=2. Full shape received: (None, 10)

In [29]:
model.fit(np.array(samples[0]), np.array(samples[1]))

InvalidArgumentError: Graph execution error:

Detected at node 'sequential_6/embedding/embedding_lookup' defined at (most recent call last):
    File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/home/florian/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/florian/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 992, in launch_instance
      app.start()
    File "/home/florian/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 711, in start
      self.io_loop.start()
    File "/home/florian/.local/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/usr/lib/python3.10/asyncio/base_events.py", line 600, in run_forever
      s